In [4]:
#pip install --upgrade pip
#pip install gspread google-auth
#pip install gspread-dataframe

In [ ]:
#acceso-a-google-sheets@my-project-lucky-pets.iam.gserviceaccount.com

In [2]:
import pandas as pd
import numpy as np
import os
import gspread
from gspread_dataframe import get_as_dataframe
import sqlite3

In [19]:
#path = os.path.abspath('../Colab Notebooks/Precios_Compra.gsheet')

gc = gspread.service_account(filename='C:/Users/Percy/AppData/Roaming/gspread/my-project-lucky-pets-c1648d0d6f20.json')
spreadsheet_V1 = gc.open("Precios")_[/]
spreadsheet_V2 = gc.open("Cantidad")
spreadsheet_V3 = gc.open("Talla")

worksheet = spreadsheet_V1.get_worksheet(3)
df_compra = get_as_dataframe(worksheet)
df_compra = df_compra.drop_duplicates()

worksheet = spreadsheet_V1.get_worksheet(2)
df_venta = get_as_dataframe(worksheet)
df_venta = df_venta.drop_duplicates()

worksheet = spreadsheet_V2.get_worksheet(2)
df_cant_comprada = get_as_dataframe(worksheet)
df_cant_comprada = df_cant_comprada.drop_duplicates()
df_cant_comprada = df_cant_comprada[~df_cant_comprada['Cantidad comprada'].isna()]
df_cant_comprada = df_cant_comprada[df_cant_comprada['Cantidad comprada']!=0]

worksheet = spreadsheet_V3.get_worksheet(2)
df_talla = get_as_dataframe(worksheet)
df_talla = df_talla.drop_duplicates()

print(df_compra.shape, df_venta.shape, df_cant_comprada.shape, df_talla.shape)

(181, 3) (181, 3) (348, 10) (11, 2)


In [5]:
df_compra.columns

Index(['Producto', 'Talla en número', 'Talla en letras', 'Precio de compra'], dtype='object')

In [7]:
df_venta.columns

Index(['Producto', 'Talla en número', 'Talla en letras', 'Precio de venta'], dtype='object')

In [10]:
pd.DataFrame(df_compra.groupby(['Producto', 'Talla en número'])['Precio de compra'].count()).reset_index().sort_values(by = 'Precio de compra', ascending = False)

,Producto,Talla en número,Precio de compra
88,Disfraz Polo Calabaza,0.0,2
162,Vestido Vintage Blonda,5.0,2
89,Disfraz Polo Calabaza,1.0,2
38,Camisa vintaje,1.0,2
25,Camisa Chanel,2.0,2
...,...,...,...
60,Chaqueta carnero a cuadros,6.0,1
61,Chaqueta carnero a cuadros,7.0,1
62,Chaqueta corduroy,0.0,1
63,Chaqueta corduroy,1.0,1


In [6]:
df_precios = df_compra.merge(df_venta, on = ['Producto', 'Talla en número', 'Talla en letras'], how = 'left')
df_precios.shape

(197, 5)

In [32]:
path = os.path.abspath('../Colab Notebooks/BD_Lucky_Pets.sqlite')
conn = sqlite3.connect(path)

df.to_sql('TABLA_PRECIOS', conn, if_exists='replace', index=False)
conn.close()